In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    print("Session runs on GOOGLE COLAB")
    !pip3 install wandb
    !git clone https://github.com/leuchtum/cse-seminar-windprognose.git
    sys.path.append('/content/cse-seminar-windprognose/prognose/')
else:
    print("Session runs on LOCAL MACHINE")

In [ ]:
import pandas as pd
import pickle
import MLForecast.networks
import MLForecast.postprocessing
import pprint

In [ ]:
!wget -O zip.zip "https://sync.luckycloud.de/f/653ae933e58a4f19a3ec/?dl=1"
!unzip -o zip.zip -d data/

In [ ]:
PATH = "data/RNNseq2vec_HORI12_HIST144_POLxy_SINGLE0/"
INDENT = "RNNseq2vec_HORI12_HIST144_POLxy_SINGLE0"
SEP = "_"
    
def load_from_others(name):
  filename = PATH + INDENT + "_" + name + ".pkl"
  with open(filename, "rb") as loadfile:
    print(f"READ IN {filename}")
    f = pickle.load(loadfile)
  return f

df_dwd = load_from_others("dwddf")
df = load_from_others("rootdf")

test_np = load_from_others("testnp")
val_np = load_from_others("valnp")
train_np = load_from_others("trainnp")

test_df = load_from_others("testdf")
val_df = load_from_others("valdf")
train_df = load_from_others("traindf")

database_report = load_from_others("report")

In [ ]:
EARLY_STOP = 8
EPOCHS = 50
TRACK_WANDB = False
SAVE = True

structure = ["LSTM_x", "DROP_d", "DENSE_y", "OUT"]
network_config = {
    "x": 64,
    "y": 64,
    "d": 0.3
}

my_model = MLForecast.networks.LSTMseq2vec(structure, network_config)
my_model.set_input_shape(train_df[0][0].shape)
my_model.set_output_shape(train_df[1][0].shape)
my_model.build()
my_model.compile()

my_model.fit(train_np, val_np, early_stop=EARLY_STOP,
             epochs=EPOCHS, track_wandb=TRACK_WANDB, save=SAVE)

my_model.load()

performance = MLForecast.postprocessing.PerformanceAnalyser(
    my_model.model, test_np)
performance_report = performance.report()

pprint.pprint({
    "structure": structure,
    "name": my_model.name,
    "config": network_config,
    "report": performance_report
})
